<a href="https://colab.research.google.com/github/Sapna1624/Car_No.Plate_Detection_Recognition/blob/main/MiniProj_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr
!pip install imutils

!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

In [ ]:
!ngrok authtoken 1wzlxLGSgg95Dh4qAZe4VPrULqu_8KTTsFJMxMFMH5rwnHSG

In [ ]:
%%writefile app.py
import streamlit as st
# import pytesseract
from PIL import Image
import cv2
from matplotlib import pyplot as plt
import numpy as np
import imutils
import easyocr
from google.colab.patches import cv2_imshow


st.title("Vehicle's Number Plate Detection")
st.header("OCR - Object Character Recognition")

img = st.sidebar.file_uploader("Choose a vehicle's image :")
if img is not None :
  img = np.array(Image.open(img))
  st.image(img, caption = "Uploaded Image")
  if st.button("PREDICT") :
    # op = pytesseract.image_to_string(img.read)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    bfilter = cv2.bilateralFilter(gray, 11, 17, 17) 
    
    edged = cv2.Canny(bfilter, 30, 200) 
    
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break
    st.write("The location points of the number plate in the image are :- ")
    st.text(location)
    # Masking the image
    mask = np.zeros(gray.shape, np.uint8)
    new_image = cv2.drawContours(mask, [location], 0,255, -1)
    st.image(new_image)
    # plt.show()
    new_image = cv2.bitwise_and(img, img, mask=mask)
    st.write("\n\n >>>Displaying the masked image of the number plate  using matplotlib :- ")
    st.image(new_image)
    st.write("\n\n >>>Displaying the masked image of the number plate  using matplotlib after making the color format as RGB:- ")
    st.image(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))
    #Cropping the image of the number plate from the masked image
    (x,y) = np.where(mask==255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2+1, y1:y2+1]
    st.write("cropped img")
    st.write("\n\n >>>Displaying the masked image of the number plate  using matplotlib after making the color format as RGB:- ")
    st.image(cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB))
    
    #Specifying the reading language to the EasyOCR as english
    reader = easyocr.Reader(['en'])

    #Reading the text from the cropped image
    result = reader.readtext(cropped_image)

    # st.write("\n \n The data obtained from the cropped image of the number plate of the vehicle, uploaded in the image : - \n ")
    # st.write(result)

    #Taking out the result i.e. only the text on the number plate
    text = result[0][-2]
    st.write("\n The text written on the number plate of the vehicle uploaded in the image : - \n")
    st.write(text)

    font = cv2.FONT_HERSHEY_SIMPLEX

    #Writing the text i.e. the output on the original image
    res = cv2.putText(img, text=text, org=(approx[0][0][0], approx[1][0][1]+60), fontFace=font, fontScale=1, color=(0,255,0), thickness=2, lineType=cv2.LINE_AA)

    #Putting a rectangle around the number plate in the original image
    res = cv2.rectangle(img, tuple(approx[0][0]), tuple(approx[2][0]), (0,255,0),3)

    st.text("\n\n >>>Displaying the result of the number plate, in the original image,  using matplotlib after making the color format as RGB:- ")
    st.image(res)


In [ ]:

!nohup streamlit run app.py &
url = ngrok.connect(port = '8501')
print(url)
